Crearé una funcion sentiment_analysis(año) que devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un analisis de sentimiento.

Importaré pandas

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('../Data/reviews.csv')

C:\Users\MI PC\AppData\Local\Temp\ipykernel_16592\1250251562.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../Data/reviews.csv')


In [4]:
df_año = df[['posted','item_id','user_review']].copy()

In [5]:
df_año

,posted,item_id,user_review
0,"Posted November 5, 2011.",1250.0,2
1,"Posted November 5, 2011.",1250.0,2
2,"Posted November 5, 2011.",1250.0,2
3,"Posted July 15, 2011.",1250.0,2
4,"Posted July 15, 2011.",1250.0,2
...,...,...,...
1423596,Posted July 20.,440.0,2
1423597,Posted July 20.,440.0,2
1423598,Posted July 2.,440.0,2
1423599,Posted July 2.,440.0,2


Elimino el punto de la columna posted

In [6]:
df_año['posted'] = df_año['posted'].str.replace('.', '')

Extraigo el año de la columna posted

In [7]:
df_año['año'] = df_año['posted'].str[-4:]

Elimino la columna posted

In [8]:
df_año= df_año.drop('posted', axis=1)

Cambio el tipo de dato de la columna año a entero

In [9]:
df_año['año'] = pd.to_numeric(df_año['año'], errors='coerce')
df_año['año'] = df_año['año'].fillna(0).astype(int)

Elimino las filas que contengan cero

In [10]:
df_año = df_año[df_año['año'] != 0]

In [11]:
df_año

,item_id,user_review,año
0,1250.0,2,2011
1,1250.0,2,2011
2,1250.0,2,2011
3,1250.0,2,2011
4,1250.0,2,2011
...,...,...,...
1423262,4000.0,2,2015
1423269,253980.0,2,2015
1423423,730.0,0,2015
1423425,730.0,2,2015


Agrupo por año y cuento los valores unicos de user_review

In [12]:
conteo = df_año.groupby('año')['user_review'].value_counts()



Imprimir los resultados

In [13]:
print(conteo)

año   user_review
2010  2                1791
      1                 771
2011  2               12709
      1                6385
      0                  92
2012  2               19534
      1               13062
      0                1056
2013  2               83378
      1               66104
      0                7444
2014  2              247258
      1              221020
      0               25386
2015  1              225241
      2              211854
      0               28193
Name: count, dtype: int64


Crearé un diccionario a partir de los datos de conteo

In [14]:
df = pd.DataFrame({
    'año': [2010, 2010,2010, 2011, 2011, 2011, 2012, 2012, 2012, 2013, 2013, 2013, 2014, 2014, 2014, 2015, 2015, 2015],
    'user_review': [2, 1,0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 1, 2, 0],
    'calificacion':[1791, 771, 0, 12709, 6385, 92, 19534, 13062, 1056, 83378, 66104, 7444, 247258, 221020, 25386, 225241, 211854, 28193]
})

In [15]:
df

,año,user_review,calificacion
0,2010,2,1791
1,2010,1,771
2,2010,0,0
3,2011,2,12709
4,2011,1,6385
5,2011,0,92
6,2012,2,19534
7,2012,1,13062
8,2012,0,1056
9,2013,2,83378


Reemplazo los valores en la columna user_review

In [16]:
df['user_review'] = df['user_review'].replace({0: 'Negative', 
                                               1: 'Neutral', 
                                               2: 'Positive'})



Imprimir el Dataframe resultante

In [17]:
print(df)

     año user_review  calificacion
0   2010    Positive          1791
1   2010     Neutral           771
2   2010    Negative             0
3   2011    Positive         12709
4   2011     Neutral          6385
5   2011    Negative            92
6   2012    Positive         19534
7   2012     Neutral         13062
8   2012    Negative          1056
9   2013    Positive         83378
10  2013     Neutral         66104
11  2013    Negative          7444
12  2014    Positive        247258
13  2014     Neutral        221020
14  2014    Negative         25386
15  2015     Neutral        225241
16  2015    Positive        211854
17  2015    Negative         28193


Crearé un csv para utilizarlo en las funciones en Fastapi

In [18]:
df.to_csv('sentiment_analysis.csv', index=False)

Por ultimo crearé la funcion

In [19]:
def sentiment_analysis(año):
    # Filtrar los datos por año y categoría
    df_filtrado1 = df[(df['año'] == año) & (df['user_review'] == 'Negative')]
    df_filtrado2= df[(df['año'] == año) & (df['user_review'] == 'Neutral')]
    df_filtrado3 = df[(df['año'] == año) & (df['user_review'] == 'Positive')]
    
    # Crear el diccionario a partir del resultado
    resultado = {
        'Negative': df_filtrado1['calificacion'].tolist()[0],
        'Neutral': df_filtrado2['calificacion'].tolist()[0],
        'Positive': df_filtrado3['calificacion'].tolist()[0]
    }
    
    return resultado

# Ejemplo de uso
print(sentiment_analysis(2014))

{'Negative': 25386, 'Neutral': 221020, 'Positive': 247258}
